In [32]:
from abc import ABC, abstractmethod

In [33]:
class Runnable(ABC):
    @abstractmethod
    def invoke(self, *args, **kwargs):
        pass

In [34]:
import random
class FakeLLM(Runnable):
    def __init__(self):
        print('LLM Created')
    
    def invoke(self, prompt):
        response_list= [
            'Dhaka is the capital of Bangladesh.',
            'BPL is the top professional football league in Bangladesh.',
            'AI stands for Artificial Intelligence.',
        ]
        return {'response':random.choice(response_list)}
        
# keep the predict method for compatibility
    def predict(self, prompt):
        response_list= [
            'Dhaka is the capital of Bangladesh.',
            'BPL is the top professional football league in Bangladesh.',
            'AI stands for Artificial Intelligence.',
        ]
        return {'response':random.choice(response_list)}

In [35]:
class FakePromptTemplate(Runnable):
    def __init__(self, template,input_variables):
        self.template = template
        self.input_variables = input_variables
        print('Prompt Template Created')

    def invoke(self, input_dict):
        return self.template.format(**input_dict)  
         
    # keep the format method for compatibility
    
    def format(self, input_dict):
        return {
                "prompt": self.template.format(**input_dict),
                **input_dict   # keep original keys
            }



In [36]:
class FakeStrOutputParser(Runnable):
    def __init__(self):
        print('String Output Parser Created')

    def invoke(self, llm_output):
        return llm_output['response']
         
    # keep the parse method for compatibility
    
    def parse(self, llm_output):
        return llm_output['response']

In [37]:
class RunnableConnector(Runnable):
    def __init__(self, runnable_list):
        self.runnable_list = runnable_list
        print('Runnable Connector Created')

    def invoke(self, input_data):
        for runnable in self.runnable_list:
            input_data = runnable.invoke(input_data)
        return input_data

In [38]:
template=FakePromptTemplate(
    template='Write a {length} poem about {topic}.',
    input_variables=['length', 'topic']
)

Prompt Template Created


In [39]:
llm=FakeLLM()

LLM Created


In [40]:
parser=FakeStrOutputParser()

String Output Parser Created


In [41]:
chain=RunnableConnector([template, llm, parser])

Runnable Connector Created


In [42]:
chain.invoke({'length':'long', 'topic':'Bangladesh'})
chain.invoke({'length':'short', 'topic':'AI'})


'AI stands for Artificial Intelligence.'

In [43]:
template=FakePromptTemplate(
    template='Write a joke about  {topic}?',
    input_variables=['topic']
)

Prompt Template Created


In [44]:
template2=FakePromptTemplate(
    template='Explain the following joke {response}',
    input_variables=['response']
)

Prompt Template Created


In [45]:
llm=FakeLLM()

LLM Created


In [46]:
parser=FakeStrOutputParser()

String Output Parser Created


In [47]:
chain1 =RunnableConnector([template, llm, parser])

Runnable Connector Created


In [48]:
chain2=RunnableConnector([ template2, llm, parser])   

Runnable Connector Created


In [57]:
class ResponseAdapter(Runnable):
    """Converts string output to dict with 'response' key"""
    def invoke(self, input_data):
        return {'response': input_data}

In [58]:
final_chain=RunnableConnector([chain1, ResponseAdapter(), chain2])

Runnable Connector Created


In [59]:
final_chain.invoke({'length':'short', 'topic':'computers'})

'AI stands for Artificial Intelligence.'